In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.axis as ax
import matplotlib.image as img
import os
%matplotlib tk

In [2]:
# modulation frequency
frequency = 10e9  # this is not practically realizable yet
#frequency = 2*10e8  # close to what's possible today
#frequency = 1.5*10e8  # realizable with our PMD camera
# speed of light
c = 3*10e8
# wavelength in meters
wavelength = c/(2*frequency)

In [3]:
# functions for generating and manipulating spatio-temporal signals

# create a sinusoidal temporal signal with the specified number of periods 
# and samples per period and return it as a column vector
def transientSignal1D(samplesPerWavelength,numWavelengths):
    vec = np.cos(np.arange(0,samplesPerWavelength*numWavelengths)*2*np.pi/samplesPerWavelength)
    return vec.reshape(len(vec),1)

# create a spatio-temporal projector signal with the specified number of pixels 
def transientSignal2D(signal1D, numPixels):
    return np.dot(np.ones((numPixels,1)), signal1D.transpose())

# apply a per-pixel shift to given spatio-temporal signal, with the shift
# expressed in units of time samples (ie. how many columns to shift a given row)
def applyDelay(signal2D, shiftVec):
    signal2Dshifted = np.zeros(signal2D.shape)
    for i in range(0,signal2D.shape[0]):
        signal2Dshifted[i,:] = np.roll(signal2D[i,:], shiftVec[i])
    return signal2Dshifted

# display a spatio-temporal signal
def displaySignal2D(signal2D, fignum, title):
    plt.figure(fignum)
    plt.xlabel(r'time $t$  ( $\times %g$ nsec)'%(secondsPerSample*10e9))
    plt.ylabel(r'pixel')
    plt.imshow(signal2D, cmap='gray')
    plt.title(title)

# convert a vector of distances to a vector of time delays
distance2time = np.vectorize(lambda x: x/c)
# convert a vector of time delays to a vector of shifts of a spatio-temporal signal 
time2sample = np.vectorize(lambda x: x/secondsPerSample)

In [4]:
# functions that implement basic geometric operations on a 2D plane

# given a source and a set of 2D points, compute the homogeneous 
# coordinates of the rays passing through the source and each of the points
#
# the rays are returned as a 3xN matrix of homogeneous ray coordinates
def joinPoints(source, points):
    # replicate the source N times to create a matrix of the same 
    # size as the 3xN matrix of points
    repSource = np.dot(source, np.ones((1,points.shape[1])))
    # the homogeneous coords of the ray through two points is given by the 
    # cross product of their homogeneous point coords
    return np.cross(repSource, points, axis=0)

# given a plane and a set of rays, compute the homogeneous 
# coordinates of the intersection of the plane and each of the rays
#
# the points are returned as a 3xN matrix of homogeneous 2D point coordinates
def intersectPlaneRays(plane, rays):
    # replicate the plane coords N times to create a matrix of the same 
    # size as the 3xN matrix of rays
    repPlane = np.dot(plane, np.ones((1,rays.shape[1])))
    # the homogeneous coords of the intersection of two rays is given by the 
    # cross product of their homogeneous ray coords
    return np.cross(repPlane, rays, axis=0)

# convert homogeneous coordinates to Euclidean coordinates
def homogeneous2euclidean(coords):
    euc = np.zeros(coords.shape)
    for i in range(0,3):
        euc[i,:] = np.divide(coords[i,:],coords[2,:])
    return euc

# compute distances between a point and an array of points
def point2pointDistance(point, points):
    dist = np.zeros((points.shape[1], 1))
    for i in range(0, points.shape[1]):
        dist[i] = np.linalg.norm(point[:,0]-points[:,i])
    return dist

In [5]:
# routines for plotting & receiving mouse input

# wait for a mouse click on the current figure and return the homogeneous 
# coordinates of the point clicked as a column vector
def ginputCoords(text='', show=False, style=None):
    plt.title(text)
    plt.show()
    coords = (plt.ginput(n=1))[0]
    if show:
        plt.plot([coords[0]], [coords[1]], style)
        plt.show()
    # we need to swap the coords because of the ways they are returned by ginput
    x, y = coords[0], coords[1]
    return np.array([x, y, 1]).reshape(3,1)

# display a set of line segments that begin at a common "source" and end
# at a set of points given by a matrix
def showSegments(source, endpoints, style='b'):
    plt.figure(1)
    for i in range(0,endpoints.shape[1]):
        x0, y0 = source[0], source[1]
        x1, y1 = endpoints[0,i], endpoints[1,i]
        plt.plot([x0, x1],[y0, y1], style)
    plt.show()

In [6]:
# routines for defining & drawing the scene

def defineScene():
    # build the scene
    fig = plt.figure(1)
    fig.clf()
    xmin = -5
    xmax = 1
    ymin = -1
    ymax = 5
    plt.plot([xmin,xmax],[0, 0],'b')
    plt.hold(True)
    plt.plot([0,0],[ymin, ymax],'b')
    plt.xlabel('x (meters)')
    plt.ylabel('y (meters)')
    plt.text(0.1,0.5,'Wall', rotation=-90, fontsize=20)
    plt.show()
    sceneBounds = [xmin, xmax, ymin, ymax]
    wallCoords = np.array([1, 0, 0]).reshape(3,1)
    return sceneBounds, wallCoords

# interactively specify the coordinates of the projector's center of projection
# and the two endpoints of the projector's plane
def defineProjector():
    msg = 'click on plot to position projector center of projection'
    projectorCenter = ginputCoords(msg, show=True, style='r+')
    plt.text(projectorCenter[0]-0.2, projectorCenter[1]-0.2, 'Projector')
    projectorEndpoints = np.zeros((3,2),dtype=np.float32)
    for i in range(0,2):
        msg = 'click on plot twice to draw the projector plane'
        endpoint = ginputCoords(msg, show=True, style='ro')
        projectorEndpoints[:,i] = endpoint[:,0]
    plt.plot(projectorEndpoints[0,:],projectorEndpoints[1,:],'r')
    return projectorCenter, projectorEndpoints

# interactively specify the coordinates of the scene point where we want the
# transient signal to arrive 'in sync'
def defineFocus():
    msg = 'click on plot to position the focus point'
    focusPoint = ginputCoords(msg, show=True, style='b+')
    plt.text(focusPoint[0]-0.2, focusPoint[1]+0.1, 'Focus')
    return focusPoint

# interactively specify the coordinates of a second scene point
def defineTest():
    msg = 'click on plot to position a second scene point'
    testPoint = ginputCoords(msg, show=True, style='b+')
    plt.text(testPoint[0]-0.2, testPoint[1]+0.1, 'Test')
    return testPoint

# interactively specify the coordinates of a second scene point
def defineRegion():
    msg = 'click on plot to specify the top-left corner of region of interest'
    topLeft = ginputCoords(msg, show=True, style='g+')
    msg = 'click on plot to specify the bottom-right corner of region of interest'
    bottomRight = ginputCoords(msg, show=True, style='g+')
    plt.plot([topLeft[0], topLeft[0]], [topLeft[1], bottomRight[1]],'g')
    plt.plot([topLeft[0], bottomRight[0]], [bottomRight[1], bottomRight[1]],'g')
    plt.plot([bottomRight[0], bottomRight[0]], [bottomRight[1], topLeft[1]],'g')
    plt.plot([bottomRight[0], topLeft[0]], [topLeft[1], topLeft[1]],'g')
    plt.text(topLeft[0]+0.2,topLeft[1]+0.1,'ROI')
    return topLeft, bottomRight


# linearly interpolate the two endpoints of the projector plane to get
# a set of N distinct projector pixels, expressed as a 3xN matrix of
# homogeneous pixel coordinates 
def projectorPixelCoords(projectorEndpoints, projectorPixels):
    pixelCoords = np.zeros((3, projectorPixels))
    for i in range(0,2):
        pixelCoords[i,:] = np.linspace(projectorEndpoints[i,0],\
                                       projectorEndpoints[i,1],\
                                       projectorPixels)
    pixelCoords[2,:] = 1
    return pixelCoords

def regionPoints(topLeft, bottomRight, spacing):
    # define a grid of points on the plane
    # this will be a 2xNxM matrix of 2D point coordinates
    # with [0,:,:] holding the Y coordinate and [1,:,:] holding the X coordinate
    # we therefore reverse the output of mgrid so that the x and y coordinates are flipped
    return (np.mgrid[bottomRight[1]:topLeft[1]:spacing, topLeft[0]:bottomRight[0]:spacing])[::-1,...]

In [7]:
# implementation of one ray-tracing step from a point source to a destination plane
# the ray sampling is expressed as a matrix of points that the rays must go through 
#
# given a source and a matrix of points, (a) cast rays that begin
# at the source and pass through each of the points in the
# matrix, (b) intersect those rays with the destination plane, and (c) return the
# homogeneous coordinates of their intersection points
def propagatePoint2Plane(source, points, destPlane):
    # compute the 2D ray associated with each point in the set
    rays = joinPoints(source, points)
    # return the intersection of these rays with the destination plane
    return homogeneous2euclidean(intersectPlaneRays(destPlane, rays))

In [8]:
# propagate a spatio-temporal signal from a point source to a destination plane
def propagateSignalPoint2Plane(signal, source, points, destPlane):
    # execute one ray-tracing step to get a set of 2D points on
    # the destination plane, in euclidean 2D coordinates
    destPoints = propagatePoint2Plane(source, points, destPlane)
    # compute the distance between the source and the destination points
    distances = point2pointDistance(source, destPoints)
    # convert these distances to shifts of the spatiotemporal transient signal
    shifts = np.int64(np.round(time2sample(distance2time(distances))))
    # shift each row of the spatiotemporal signal to account for light arrival delays
    return applyDelay(signal, shifts), shifts, destPoints

# propagate a spatio-temporal signal from an area source to a destination point,
# with the area source expressed as an array of homogeneous point coordinates
def propagateSignalAreaSource2Point(signal, sourcePoints, destPoint):
    # compute the distance between the source points and the destination point
    distances = point2pointDistance(destPoint, sourcePoints)
    # convert these distances to shifts of the spatiotemporal transient signal
    shifts = np.int64(np.round(time2sample(distance2time(distances))))
    # shift each row of the spatiotemporal signal to account for light arrival delays
    return applyDelay(signal, shifts), shifts

# propagate a spatio-temporal signal from a point source to a destination point
# via an intermediate diffuse reflection plane. the rays used for propagation are
# expressed as an array of points that the rays from the source must go through
def propagateSignalPoint2Point(signal, source, points, dest, intermediatePlane):
    # first we compute the spatio-temporal signal after one ray-tracing step
    planeSignal, source2planeDelays, planeCoords = \
        propagateSignalPoint2Plane(signal, source, points, intermediatePlane)
    # now we compute the spatio-temporal signal after a second ray-tracing step,
    # from the intermediate plane to the destination point
    destSignal, plane2destDelays = \
        propagateSignalAreaSource2Point(planeSignal, planeCoords, dest)
    # return all the computed quantities
    return destSignal, planeSignal, source2planeDelays, plane2destDelays, planeCoords


In [9]:
### Simulation parameters

In [10]:
# string to use for saving results of this simulation run
runID = 'example4'

In [11]:
# number of projector pixels
# increasing the number of pixels results in better 
# localization of the focus spot but makes the run much
# more computationally intensive
# projectorPixels = 200  # this is a pretty high setting
projectorPixels = 10 # use this initially for testing/exploring 

In [12]:
# samples to include in the space-time plots of intensity distribution
samplesPerWavelength = 25
numWavelengths = 4
metersPerSample = wavelength/samplesPerWavelength
secondsPerSample = metersPerSample/c
numSamples = samplesPerWavelength * numWavelengths
print 'Meters per time sample = %g'%(metersPerSample)

Meters per time sample = 0.006


In [13]:
### Defining the scene

In [14]:
# specify basic scene parameters interactively
plt.figure(1)
plt.axis('equal')
sceneBounds, wallCoords = defineScene()
projectorCenter, projectorEndpoints = defineProjector()
focusPoint = defineFocus()
testPoint = defineTest()
topLeft, bottomRight = defineRegion()
# compute the 2D point associated with each projector pixel
pcoords = projectorPixelCoords(projectorEndpoints,projectorPixels)

C:\Program Files\Anaconda2\lib\site-packages\matplotlib\backend_bases.py:2437: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [15]:
### Do the ray-tracing simulation

In [16]:
# define & display the projector's spatio-temporal signal
projectorSignal = transientSignal2D(transientSignal1D(samplesPerWavelength,numWavelengths),\
                                    projectorPixels)
displaySignal2D(projectorSignal, 2, 'Transient signal at projector')

In [17]:
# first, we execute the ray tracing from the source to the focus point and display the light paths
focusSignal, wallSignal, source2wallDelays, wall2focusDelays, wallPointCoords = \
    propagateSignalPoint2Point(projectorSignal, projectorCenter, pcoords, \
                               focusPoint, wallCoords)
# show the paths from the projector center to the wall
showSegments(projectorCenter, wallPointCoords)
# show the paths from the wall to the focus point
showSegments(focusPoint, wallPointCoords)
# display the spatio-temporal signal received at the wall
displaySignal2D(wallSignal, 3, 'Transient signal at wall')
# display the spatio-temporal signal received at the focus point
displaySignal2D(focusSignal, 4, 'Transient signal at focus')

In [18]:
# now that we can compute the total delays to the focus point, we can 
# apply the reverse delays to the initial signal so it arrives
# 'in sync' at the focus point
totalDelays = source2wallDelays + wall2focusDelays
focusedProjectorSignal = applyDelay(projectorSignal, -totalDelays)
# let's display this signal as well
displaySignal2D(focusedProjectorSignal, 5, 'Re-focused transient signal at projector')
# propagating this signal should produce a spatio-temporal signal that arrives 'in sync'
# at the focus point
focusSignal2, wallSignal, source2wallDelays, wall2focusDelays, wallPointCoords = \
    propagateSignalPoint2Point(focusedProjectorSignal, projectorCenter, pcoords, \
                               focusPoint, wallCoords)
displaySignal2D(focusSignal2, 6, 'New transient signal at focus')
# the transient signal arriving at the focus point across all paths is given
# by the column-wise sum 
focusSignal1D = focusSignal2.sum(axis=0)

In [19]:
# now let's propagate the signal to a second 'test' point in the scene
# the signal should not arrive 'in sync' for such a point
testSignal, wallSignal, source2wallDelays, wall2focusDelays, wallPointCoords = \
    propagateSignalPoint2Point(focusedProjectorSignal, projectorCenter, pcoords, \
                               testPoint, wallCoords)
displaySignal2D(testSignal, 7, 'Transient signal at test point')
# the transient signal arriving at the test point across all paths is given
# by the column-wise sum 
testSignal1D = testSignal.sum(axis=0)
plt.figure(8)
plt.plot(focusSignal1D,'r')
plt.hold(True)
plt.plot(testSignal1D,'b')
plt.title('Comparison of transient signal arriving at focus point and test point')
plt.legend((r'focus point $(%0.3g,%0.3g)$'%(focusPoint[0],focusPoint[1]),\
            r'test point $(%0.3g,%0.3g)$'%(testPoint[0],testPoint[1])))
plt.hold(False)

In [20]:
# finally, let's do the calculation for a whole region of points on the scene plane
# this is done very inefficiently at the moment, for the sake of simplicity..

# we first discretize the region of interest at the resolution of individual time samples
# this returns an MxN grid of 2D euclidean coordinates
rpoints = regionPoints(topLeft, bottomRight, metersPerSample)
# now let's allocate the array that will hold the spatio-temporal signals of all the points
# this will be an TxMxN array where T is the number of time samples
regionSignals = np.zeros((rpoints.shape[1], rpoints.shape[2], numSamples))
print 'Grid size is %dx%d'%(rpoints.shape[1], rpoints.shape[2])

Grid size is 27x103


In [21]:
# finally, we iterate over all points, performing the ray-tracing calculations
for i in range(0,rpoints.shape[1]):
    # iterate over x coordinates
    for j in range(0,rpoints.shape[2]):
        # iterate over y coordinates
        testPoint[0] = rpoints[0,i,j]
        testPoint[1] = rpoints[1,i,j]
        testSignal, wallSignal, source2wallDelays, wall2focusDelays, wallPointCoords = \
                    propagateSignalPoint2Point(focusedProjectorSignal, projectorCenter, \
                                                pcoords, testPoint, wallCoords)
        regionSignals[i,j,:] = testSignal.sum(axis=0)

In [22]:
### Show the results

In [23]:
plt.figure(9)
asp = (topLeft[0,0]-bottomRight[0,0])/(bottomRight[1,0]-topLeft[1,0])
# to display using the imshow function, we need to flip the rows
# of the matrix because imshow displays images with the (0,0) pixel
# in the upper-left rather than lower-left corner
plt.imshow(np.squeeze(regionSignals[:,:,0])[::-1],\
           cmap='gray', aspect=asp,\
           extent=(topLeft[0,0],bottomRight[0,0],bottomRight[1,0],topLeft[1,0]))
plt.title('Intensity distribution in region of interest')
plt.xlabel('x (meters)')
plt.ylabel('y (meters)')
plt.hold(True)
plt.plot(focusPoint[0,0],focusPoint[1,0],'r+')
plt.text(focusPoint[0,0]-0.1,focusPoint[1,0]+0.1,'Focus')
plt.hold(False)

C:\Program Files\Anaconda2\lib\site-packages\matplotlib\axes\_base.py:1210: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if aspect == 'normal':
C:\Program Files\Anaconda2\lib\site-packages\matplotlib\axes\_base.py:1215: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  elif aspect in ('equal', 'auto'):


In [24]:
### Save the basic parameters and results

In [25]:
# create a directory for the results if it doesn't exist
if not os.path.exists(runID):
    os.mkdir(runID)
# save the interactively-specified variables
np.savez_compressed(runID+'/'+'scene_params.npz',\
                    sceneBounds=sceneBounds,\
                    wallCoords=wallCoords,\
                    projectorCenter=projectorCenter,\
                    projectorEndpoints=projectorEndpoints,\
                    focusPoint=focusPoint,\
                    topLeft=topLeft,\
                    bottomRight=bottomRight)
# save the figures in PDF format
plt.figure(1)
plt.savefig(runID+'/'+'scene.pdf')
plt.figure(2)
plt.savefig(runID+'/'+'transient.unfocused.projector.pdf')
plt.figure(3)
plt.savefig(runID+'/'+'transient.unfocused.wall.pdf')
plt.figure(4)
plt.savefig(runID+'/'+'transient.unfocused.focusPoint.pdf')
plt.figure(5)
plt.savefig(runID+'/'+'transient.focused.projector.pdf')
plt.figure(6)
plt.savefig(runID+'/'+'transient.focused.focusPoint.pdf')
plt.figure(7)
plt.savefig(runID+'/'+'transient.focused.testPoint.pdf')
plt.figure(8)
plt.savefig(runID+'/'+'comparison.pdf')
plt.figure(9)
plt.savefig(runID+'/'+'distribution.pdf')
# save all basic matrices
np.savez_compressed(runID+'/'+'results.npz',\
                     totalDelays=totalDelays,\
                     focusedProjectorSignal=focusedProjectorSignal,\
                     focusSignal2=focusSignal2,\
                     wallSignal=wallSignal,\
                     source2wallDelays=source2wallDelays,\
                     wall2focusDelays=wall2focusDelays,\
                     wallPointCoords=wallPointCoords,\
                     focusSignal1D=focusSignal1D,\
                     rpoints=rpoints,\
                     regionSignals=regionSignals)

In [26]:
# loading & accessing the saved results
qq = np.load(runID+'/'+'scene_params.npz')
focusPoint = qq['focusPoint']
focusPoint

array([[-1.625  ],
       [ 0.40625],
       [ 1.     ]])